In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [2]:
import torch
import pandas as pd
import numpy as np

In [3]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.1 MB/s eta 0:00:00


In [4]:
df = pd.read_csv('/content/Complaint data annotation (explain)_updated - cd (1).csv')

In [5]:
df.keys()

Index(['id', 'tweet', 'label', 'domain', 'sentiment', 'emotion', 'Severity',
       'Explain'],
      dtype='object')

In [6]:
df.head()

,id,tweet,label,domain,sentiment,emotion,Severity,Explain
0,1,@FC_HELP can I return online purchases to a Ho...,0,apparel,Neutral,other,0,can I return online purchases to a House of Fr...
1,2,@FC_Help Hi - I'm writing a piece for MSN Him ...,0,apparel,Positive,other,0,Hi - I'm writing a piece for MSN Him and wonde...
2,3,@FC_Help i need to check my order,0,apparel,Neutral,other,0,i need to check my order
3,4,@FC_Help I need to get in contact with someone...,1,apparel,Neutral,other,1,I need to get in contact with someone regardin...
4,5,@FC_Help How can I get a hold of you so we can...,0,apparel,Negative,other,0,How can I get a hold of you so we can discuss ...


In [7]:
domain_dict = {
    "other" : 0,
    "services" : 1,
    "random_reply" : 2,
    "software" : 3,
    "retail" : 4,
    "random_tweet" : 5,
    "transport" : 6,
    "cars" : 7,
    "food" : 8,
    "apparel" : 9,
    "electronics" : 10
}

# note to take string lower
senti_dict = {
    'negative' : 0,
    'positive' : 1,
    'neutral' : 2
}

emo_dict = {
    'sadness' : 0, 
    'joy' : 1, 
    'other' : 2, 
    'anger' : 3, 
    'disgust' : 4, 
    'surprise' : 5, 
    'fear' : 6
}

In [8]:
label = []
domain = []
emotion = []
sentiment = []
severity = []
explain = []

for i in range(len(df)):
  if (pd.isna(df['label'][i]) or pd.isna(df['domain'][i]) or pd.isna(df['emotion'][i]) or pd.isna(df['sentiment'][i]) or pd.isna(df['Severity'][i]) or pd.isna(df['Explain'][i])):
    continue
  label.append(df['label'][i])
  domain.append(domain_dict[df['domain'][i]])
  emotion.append(emo_dict[df['emotion'][i]])
  sentiment.append(senti_dict[(df['sentiment'][i]).lower()])
  severity.append(df['Severity'][i])
  explain.append(df['Explain'][i])

In [9]:
from sklearn.model_selection import train_test_split

label_train, label_test, domain_train, domain_test, emotion_train, emotion_test, sentiment_train, sentiment_test, severity_train, severity_test, explain_train, explain_test = train_test_split(label, domain, emotion, sentiment, severity, explain, test_size = 0.2, random_state = 42, shuffle = True)
label_train, label_val, domain_train, domain_val, emotion_train, emotion_val, sentiment_train, sentiment_val, severity_train, severity_val, explain_train, explain_val = train_test_split(label_train, domain_train, emotion_train, sentiment_train, severity_train, explain_train, test_size = 0.2, random_state = 42, shuffle = True)

In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(text):
  g_input_ids = []
  g_attention_mask = []

  for line in text:
    inputs = tokenizer(line, return_tensors="pt")

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    size = input_ids.shape[1]

    if size < 60:
      input_ids = torch.cat((input_ids, torch.zeros(1, 60-size)), dim = 1)
      attention_mask = torch.cat((attention_mask, torch.zeros(1, 60-size)), dim = 1)
    else:
      input_ids = input_ids[:, :60]
      attention_mask = attention_mask[:, :60]
    
    # Why we use np.array()
    g_input_ids.append(np.array(input_ids.reshape(-1)))
    g_attention_mask.append(np.array(attention_mask.reshape(-1)))

  g_input_ids = torch.tensor(g_input_ids)
  g_attention_mask = torch.tensor(g_attention_mask)

  return g_input_ids, g_attention_mask

In [11]:
input_ids_train, attention_mask_train = preprocess_data(explain_train)
input_ids_val, attention_mask_val = preprocess_data(explain_val)
input_ids_test, attention_mask_test = preprocess_data(explain_test)

<ipython-input-10-eb1c066b164e>:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  g_input_ids = torch.tensor(g_input_ids)


In [12]:
class Complaint_dataset():
  def __init__(self, input_ids, attention_mask, label, domain, emotion, sentiment, severity):
    self.input_ids = input_ids
    self.attention_mask = attention_mask
    self.label = label
    self.domain = domain
    self.emotion = emotion
    self.sentiment = sentiment
    self.severity = severity
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    
    sample = {
        "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
        "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),
        "label" : torch.tensor(self.label[idx]).long().to(device),
        "domain" : torch.tensor(self.domain[idx]).long().to(device),
        "emotion" : torch.tensor(self.emotion[idx]).long().to(device),
        "sentiment" : torch.tensor(self.sentiment[idx]).long().to(device),
        "severity" : torch.tensor(self.severity[idx]).long().to(device)
    }

    return sample

In [13]:
from torch.utils.data import DataLoader

complaint_train = Complaint_dataset(input_ids_train, attention_mask_train, label_train, domain_train, emotion_train, sentiment_train, severity_train)
train_dataloader = DataLoader(complaint_train, batch_size = 32, shuffle = False)

complaint_val = Complaint_dataset(input_ids_val, attention_mask_val, label_val, domain_val, emotion_val, sentiment_val, severity_val)
val_dataloader = DataLoader(complaint_val, batch_size = 32, shuffle = False)

complaint_test = Complaint_dataset(input_ids_test, attention_mask_test, label_test, domain_test, emotion_test, sentiment_test, severity_test)
test_dataloader = DataLoader(complaint_test, batch_size = 32, shuffle = False)

In [14]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

# What is super(Complaint) doing and why the name __init__() as such
# Practise OOPS

class Complaint(nn.Module):
  def __init__(self):
    super(Complaint, self).__init__()
    self.bert_model = BertModel.from_pretrained('bert-base-uncased')
    self.complaint = nn.Linear(768, 2)
    self.severity = nn.Linear(768, 5)
    self.emotion = nn.Linear(768, 7)
    self.sentiment = nn.Linear(768, 3)
  def forward(self, input_ids, attention_mask):
    data = self.bert_model(input_ids = input_ids, attention_mask = attention_mask).pooler_output
    Complaint = self.complaint(data)
    Severity = self.severity(data)
    Emotion = self.emotion(data)
    Sentiment = self.sentiment(data)
    return data, Complaint, Emotion, Sentiment, Severity

In [15]:
import torch.nn.functional as F

class SupConLoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()
        self.alpha = 0.5
        self.temp = 0.1

    def nt_xent_loss(self, anchor, target, labels):
        with torch.no_grad():
            labels = labels.unsqueeze(-1)
            mask = torch.eq(labels, labels.transpose(0, 1))
            mask = mask ^ torch.diag_embed(torch.diag(mask))

        anchor_dot_target = torch.einsum('bd,cd->bc', anchor, target) / self.temp

        anchor_dot_target = anchor_dot_target - torch.diag_embed(torch.diag(anchor_dot_target))

        logits_max, _ = torch.max(anchor_dot_target, dim=1, keepdim=True)
        logits = anchor_dot_target - logits_max.detach()

        exp_logits = torch.exp(logits)

        logits = logits * mask
        log_prob = logits - torch.log(exp_logits.sum(dim=1, keepdim=True) + 1e-12)

        mask_sum = mask.sum(dim=1)
        mask_sum = torch.where(mask_sum == 0, torch.ones_like(mask_sum), mask_sum)

        pos_logits = (mask * log_prob).sum(dim=1) / mask_sum.detach()
        loss = -1 * pos_logits.mean()
        return loss

    def forward(self, output, predicted, targets):
        normed_output = F.normalize(output, dim=-1)
        ce_loss = (1 - self.alpha) * self.xent_loss(predicted, targets)
        cl_loss = self.alpha * self.nt_xent_loss(normed_output, normed_output, targets)
        return cl_loss + ce_loss

In [16]:
complaint_model = Complaint()
complaint_model = complaint_model.to(device)
complaint_model.train()

loss_func = SupConLoss()
optimizer = torch.optim.Adam(complaint_model.parameters(), lr = 1e-5, weight_decay = 0)
epochs = 20
min_val_loss = 20

complaint_model.train()

for i in range(epochs):

  for data in train_dataloader:
    complaint_model.zero_grad()
    comp_train = data["label"].to(device)
    sev_train = data["severity"].to(device)
    emo_train = data["emotion"].to(device)
    senti_train = data["sentiment"].to(device)
    data, comp_out, emo_out, senti_out, sev_out = complaint_model.forward(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

    loss_train = loss_func(data, comp_out, comp_train) + loss_func(data, sev_out, sev_train) + loss_func(data, emo_out, emo_train) + loss_func(data, senti_out, senti_train)
    loss_train.backward()
    optimizer.step()

    complaint_model.eval()

    with torch.no_grad():
      total_loss_val = 0
      for data in val_dataloader:
        comp_val = data["label"].to(device)
        sev_val = data["severity"].to(device)
        emo_val = data["emotion"].to(device)
        senti_val = data["sentiment"].to(device)
        data_val, comp_val_out, emo_val_out, senti_val_out, sev_val_out = complaint_model(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

        val_loss = loss_func(data_val, comp_val_out, comp_val) + loss_func(data_val, sev_val_out, sev_val) + loss_func(data_val, emo_val_out, emo_val) + loss_func(data_val, senti_val_out, senti_val)

        total_val = comp_val.size(0)
        total_loss_val += val_loss.item()

      val_loss = total_loss_val / total_val

    if ((min_val_loss-val_loss) > 1e-4):
      min_val_loss = val_loss
      torch.save(complaint_model.state_dict(), "bert_model.pt")

  print(f"Epoch : {i+1}")
  print(f"Validation loss : {val_loss}")

  complaint_model.load_state_dict(torch.load("bert_model.pt"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-12-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.cl

Epoch : 1
Validation loss : 19.775026559829712
Epoch : 2
Validation loss : 18.81733250617981
Epoch : 3
Validation loss : 18.282465398311615
Epoch : 4
Validation loss : 18.06517219543457
Epoch : 5
Validation loss : 18.097975850105286
Epoch : 6
Validation loss : 18.201288759708405
Epoch : 7
Validation loss : 18.104013323783875
Epoch : 8
Validation loss : 17.967324316501617
Epoch : 9
Validation loss : 17.886028468608856
Epoch : 10
Validation loss : 17.82459980249405
Epoch : 11
Validation loss : 17.754263043403625
Epoch : 12
Validation loss : 17.785349786281586
Epoch : 13
Validation loss : 17.81677496433258
Epoch : 14
Validation loss : 17.696775794029236
Epoch : 15
Validation loss : 17.926905393600464
Epoch : 16
Validation loss : 17.66270399093628
Epoch : 17
Validation loss : 17.697213411331177
Epoch : 18
Validation loss : 17.711541712284088
Epoch : 19
Validation loss : 17.659736275672913
Epoch : 20
Validation loss : 17.73404633998871


In [17]:
from sklearn.metrics import *

In [18]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 23.3 MB/s eta 0:00:00


In [19]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryF1Score, BinaryRecall
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassF1Score, MulticlassRecall

comp_accuracy = BinaryAccuracy()
comp_precision = BinaryPrecision()
comp_f1 = BinaryF1Score()
comp_recall = BinaryRecall() 

sev_accuracy = MulticlassAccuracy(num_classes=5)
sev_precision = MulticlassPrecision(num_classes=5)
sev_f1 = MulticlassF1Score(num_classes=5)
sev_recall = MulticlassRecall(num_classes=5)

emo_accuracy = MulticlassAccuracy(num_classes=7)
emo_precision = MulticlassPrecision(num_classes=7)
emo_f1 = MulticlassF1Score(num_classes=7)
emo_recall = MulticlassRecall(num_classes=7)

senti_accuracy = MulticlassAccuracy(num_classes=3)
senti_precision = MulticlassPrecision(num_classes=3)
senti_f1 = MulticlassF1Score(num_classes=3)
senti_recall = MulticlassRecall(num_classes=3)

with torch.no_grad():
    for data in test_dataloader:
      lab_comp = data["label"]
      lab_emo = data["emotion"]
      lab_senti = data["sentiment"]
      lab_sev = data["severity"]

      _, comp_out, emo_out, senti_out, sev_out = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])

      _, pred_comp = torch.max(comp_out.data,1)
      _, pred_emo = torch.max(emo_out.data,1)
      _, pred_senti = torch.max(senti_out.data,1)
      _, pred_sev = torch.max(sev_out.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu()) 
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())

      emo_accuracy.update(pred_emo.cpu(), lab_emo.cpu())
      emo_precision.update(pred_emo.cpu(), lab_emo.cpu())
      emo_f1.update(pred_emo.cpu(), lab_emo.cpu())
      emo_recall.update(pred_emo.cpu(), lab_emo.cpu())
      emo_test_accuracy = emo_accuracy.compute()

      senti_accuracy.update(pred_senti.cpu(), lab_senti.cpu())
      senti_precision.update(pred_senti.cpu(), lab_senti.cpu())
      senti_f1.update(pred_senti.cpu(), lab_senti.cpu())
      senti_recall.update(pred_senti.cpu(), lab_senti.cpu())
      senti_test_accuracy = senti_accuracy.compute()

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    emo_test_accuracy = emo_accuracy.compute()
    emo_test_precision = emo_precision.compute()
    emo_test_f1 = emo_f1.compute()
    emo_test_recall = emo_recall.compute()

    senti_test_accuracy = senti_accuracy.compute()
    senti_test_precision = senti_precision.compute()
    senti_test_f1 = senti_f1.compute()
    senti_test_recall = senti_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}") 

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}") 

    print(f"emo_test_accuracy : {emo_test_accuracy}")
    print(f"emo_test_precision : {emo_test_precision}")
    print(f"emo_test_f1 : {emo_test_f1}")
    print(f"emo_test_recall : {emo_test_recall}")

    print(f"senti_test_accuracy : {senti_test_accuracy}")
    print(f"senti_test_precision : {senti_test_precision}")
    print(f"senti_test_f1 : {senti_test_f1}")
    print(f"senti_test_recall : {senti_test_recall}")

<ipython-input-12-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
<ipython-input-12-aa0febf124e5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),


comp_test_accuracy : 0.865217387676239
comp_test_precision : 0.8464912176132202
comp_test_f1 : 0.8058454990386963
comp_test_recall : 0.7689242959022522
sev_test_accuracy : 0.320623517036438
sev_test_precision : 0.3055311143398285
sev_test_f1 : 0.3082727789878845
sev_test_recall : 0.320623517036438
emo_test_accuracy : 0.3630354404449463
emo_test_precision : 0.3859032094478607
emo_test_f1 : 0.36892345547676086
emo_test_recall : 0.3630354404449463
senti_test_accuracy : 0.6666310429573059
senti_test_precision : 0.6664936542510986
senti_test_f1 : 0.665217399597168
senti_test_recall : 0.6666310429573059


In [20]:
import numpy as np
import torch
import pandas as pd

In [21]:
df = pd.read_excel(r'/content/aspect_complain_formatted.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [22]:
df.head()

,Domain,Complaint/ Opinion,Over-all_Complaint Label,Aspects_1,Complaint_1,Aspect_2,Complaint_2,Aspect_3,Complaint_3,Complaint_Cause,Severity level,Sentiment,Emotion
0,Customer Service,https://consumercomplaintscourt.com/ifb-washin...,1,technical issue,1.0,NaN,NaN,NaN,NaN,IFB washing machine,accusation,negative,anger
1,Credit card,@AxisBank @AxisBankSupport Great policy to can...,1,Feedback,0.0,NaN,NaN,NaN,NaN,Great policy to cancel credit card application...,Accusation,negative,anger
2,Credit card,@Uber_Support But don’t worry cause then after...,1,Payment,0.0,NaN,NaN,NaN,NaN,I was missing more than $5 worth of food so I...,Accusation,negative,anger
3,Credit card,"@RTGameCrowd @TeamYouTube Great, now i cant ev...",1,payment issue,0.0,NaN,NaN,NaN,NaN,not giving youtube my credit card or id info t...,Accusation,negative,anger
4,Credit card,Just great… BMO has blocked my credit card and...,1,payment issue,0.0,NaN,NaN,NaN,NaN,BMO has blocked my credit card,Accusation,negative,anger


In [23]:
domain = set()
severity = set()
sentiment = set()
emotion = set()
complaint = set()
aspect = set()
complaint_1 = set()

In [24]:
for i in range(len(df)):
  domain.add(df['Domain'][i])
  severity.add(df['Severity level'][i])
  sentiment.add(df['Sentiment'][i])
  emotion.add(df['Emotion'][i])
  complaint.add(df['Over-all_Complaint Label'][i])
  aspect.add(df['Aspects_1'][i])
  complaint_1.add(df['Complaint_1'][i])

In [25]:
print(domain)
print(sentiment)
print(severity)
print(emotion)
print(aspect)
print(complaint)
print(complaint_1)

{'Customer Service', 'Salary', ' Retail Banking ', 'Credit card', 'Investments', 'Debit Card', 'Financial Policies  ', 'Retail Banking', ' Customer Service', 'Economy', 'Credit Card', 'Financial Policies', 'Retail banking', 'Loan', 'Transactional Deficiency'}
{'negative', 'neutral', 'Neutral', 'positive'}
{'Accusation', 'accusation', 'Blame', 'Non-Complaint', 'No explicit reproach', 'Disapproval'}
{'surprise', 'Anger', 'disgust', 'anger', 'happiness', 'Sadness', 'fear', 'other', 'sadness'}
{'net banking', 'personal opinion,', 'genral query', ' news ', 'Delay response', 'payment', 'financial policy', 'Miscellaneous inforation', 'news ', 'Financial Policies', 'Feeedback', 'General query', 'Financial policies', 'Financial Policy', 'delay response', 'bank issue, harassment', 'crdentials', 'Miscellaneous information', 'Finacial policy', 'bankinhg', 'crdentials ', 'financial gain', 'credentials ', ' payment', 'misellaneous information', 'misellaneous infromation', 'Payment ', 'credentials is

In [26]:
senti_dict = {
    'negative' : 0,
    'positive' : 1,
    'neutral' : 2
}

emo_dict = {
    'sadness' : 0, 
    'happiness' : 1, 
    'other' : 2, 
    'anger' : 3, 
    'disgust' : 4, 
    'surprise' : 5, 
    'fear' : 6
}

sev_dict = {
    'non-complaint' : 0,
    'no explicit reproach' : 1, 
    'disapproval' : 2,
    'accusation' : 3, 
    'blame' : 4, 
}

In [27]:
df.keys()

Index(['Domain', 'Complaint/ Opinion', 'Over-all_Complaint Label', 'Aspects_1',
       'Complaint_1', 'Aspect_2', 'Complaint_2', 'Aspect_3', 'Complaint_3',
       'Complaint_Cause', 'Severity level', 'Sentiment', 'Emotion'],
      dtype='object')

In [28]:
label = []
domain = []
emotion = []
sentiment = []
severity = []
text = []

for i in range(len(df)):
  if (pd.isna(df['Over-all_Complaint Label'][i]) or pd.isna(df['Domain'][i]) or pd.isna(df['Emotion'][i]) or pd.isna(df['Sentiment'][i]) or pd.isna(df['Severity level'][i]) or pd.isna(df['Complaint/ Opinion'][i])):
    continue
  text.append(df['Complaint/ Opinion'][i])
  domain.append(df['Domain'][i])
  severity.append(sev_dict[df['Severity level'][i].lower()])
  sentiment.append(senti_dict[df['Sentiment'][i].lower()])
  emotion.append(emo_dict[df['Emotion'][i].lower()])
  label.append(df['Over-all_Complaint Label'][i])

In [39]:
from sklearn.model_selection import train_test_split

_, label_test, _, domain_test, _, emotion_test, _, sentiment_test, _, severity_test, _, text_test = train_test_split(label, domain, emotion, sentiment, severity, text, test_size = 0.2, random_state = 42, shuffle = True)

In [45]:
class Complaint_dataset():
  def __init__(self, input_ids, attention_mask, label, domain, emotion, sentiment, severity):
    self.input_ids = input_ids
    self.attention_mask = attention_mask
    self.label = label
    self.domain = domain
    self.emotion = emotion
    self.sentiment = sentiment
    self.severity = severity
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    sample = {
        "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
        "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),
        "label" : torch.tensor(self.label[idx]).long().to(device),
        "emotion" : torch.tensor(self.emotion[idx]).long().to(device),
        "sentiment" : torch.tensor(self.sentiment[idx]).long().to(device),
        "severity" : torch.tensor(self.severity[idx]).long().to(device)
    }

    return sample

In [40]:
print(text_test)

['@GlobalCrypto12 @tapfantasy2021 Is blockchain gaming, people are always worried about the transaction speed because they worry about their transaction going to fail. How fast and secure is DNA×CAT  gaming platform? If the number of users spikes, will transactions on DNA×CAT get congested?', "China's economy grew 8  in 2021 but property, virus threats loom: AFP poll https://t.co/X7Ljn4bf03 via @economictimes", 'Find all Deltron products here at KGA Enclosures.\n» Great Low Prices\n» Fast Delivery Times\n» Pay via - BACS/PayPal/Credit Card\nhttps://t.co/iPxBpjhXCD\n\n#ManufacturingUK #electronics #UKmfg #MFG #GBmfg #MadeInAmerica #MadeInBritain #engineering #manufacturing #Manufacturers https://t.co/DrNXQS5luF', 'One more kind of fraud is done by  @Flipkart Today. without delivering the order status marked as delivered. Now, I got stuck b/w CC &amp; theirs resolution. My time and money both are endangered. order ID OD223833304318217000. \n\n@flipkartsupport', '@HDFCBank_Cares I want to

In [41]:
input_ids_test, attention_mask_test = preprocess_data(text_test)

In [46]:
from torch.utils.data import DataLoader

complaint_test = Complaint_dataset(input_ids_test, attention_mask_test, label_test, domain_test, emotion_test, sentiment_test, severity_test)
test_dataloader = DataLoader(complaint_test, batch_size = 32, shuffle = False)

In [44]:
print(test_dataloader)

In [47]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryF1Score, BinaryRecall
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassF1Score, MulticlassRecall

comp_accuracy = BinaryAccuracy()
comp_precision = BinaryPrecision()
comp_f1 = BinaryF1Score()
comp_recall = BinaryRecall() 

sev_accuracy = MulticlassAccuracy(num_classes=5)
sev_precision = MulticlassPrecision(num_classes=5)
sev_f1 = MulticlassF1Score(num_classes=5)
sev_recall = MulticlassRecall(num_classes=5)

emo_accuracy = MulticlassAccuracy(num_classes=7)
emo_precision = MulticlassPrecision(num_classes=7)
emo_f1 = MulticlassF1Score(num_classes=7)
emo_recall = MulticlassRecall(num_classes=7)

senti_accuracy = MulticlassAccuracy(num_classes=3)
senti_precision = MulticlassPrecision(num_classes=3)
senti_f1 = MulticlassF1Score(num_classes=3)
senti_recall = MulticlassRecall(num_classes=3)

with torch.no_grad():
    for data in test_dataloader:
      lab_comp = data["label"]
      lab_emo = data["emotion"]
      lab_senti = data["sentiment"]
      lab_sev = data["severity"]

      _, comp_out, emo_out, senti_out, sev_out = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])

      _, pred_comp = torch.max(comp_out.data,1)
      _, pred_emo = torch.max(emo_out.data,1)
      _, pred_senti = torch.max(senti_out.data,1)
      _, pred_sev = torch.max(sev_out.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu()) 
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())

      emo_accuracy.update(pred_emo.cpu(), lab_emo.cpu())
      emo_precision.update(pred_emo.cpu(), lab_emo.cpu())
      emo_f1.update(pred_emo.cpu(), lab_emo.cpu())
      emo_recall.update(pred_emo.cpu(), lab_emo.cpu())
      emo_test_accuracy = emo_accuracy.compute()

      senti_accuracy.update(pred_senti.cpu(), lab_senti.cpu())
      senti_precision.update(pred_senti.cpu(), lab_senti.cpu())
      senti_f1.update(pred_senti.cpu(), lab_senti.cpu())
      senti_recall.update(pred_senti.cpu(), lab_senti.cpu())
      senti_test_accuracy = senti_accuracy.compute()

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    emo_test_accuracy = emo_accuracy.compute()
    emo_test_precision = emo_precision.compute()
    emo_test_f1 = emo_f1.compute()
    emo_test_recall = emo_recall.compute()

    senti_test_accuracy = senti_accuracy.compute()
    senti_test_precision = senti_precision.compute()
    senti_test_f1 = senti_f1.compute()
    senti_test_recall = senti_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}") 

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}") 

    print(f"emo_test_accuracy : {emo_test_accuracy}")
    print(f"emo_test_precision : {emo_test_precision}")
    print(f"emo_test_f1 : {emo_test_f1}")
    print(f"emo_test_recall : {emo_test_recall}")

    print(f"senti_test_accuracy : {senti_test_accuracy}")
    print(f"senti_test_precision : {senti_test_precision}")
    print(f"senti_test_f1 : {senti_test_f1}")
    print(f"senti_test_recall : {senti_test_recall}")

<ipython-input-45-5344bf8ee05c>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
<ipython-input-45-5344bf8ee05c>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),


comp_test_accuracy : 0.7959814667701721
comp_test_precision : 0.8455149531364441
comp_test_f1 : 0.7940717339515686
comp_test_recall : 0.7485294342041016
sev_test_accuracy : 0.3908873200416565
sev_test_precision : 0.3756318688392639
sev_test_f1 : 0.3577946424484253
sev_test_recall : 0.3908873200416565
emo_test_accuracy : 0.2715338468551636
emo_test_precision : 0.2738591432571411
emo_test_f1 : 0.21873721480369568
emo_test_recall : 0.2715338468551636
senti_test_accuracy : 0.5834211111068726
senti_test_precision : 0.5698529481887817
senti_test_f1 : 0.5664278268814087
senti_test_recall : 0.5834211111068726
